In [ ]:
from ../scripts.dataset_creation import create_final_df

In [ ]:
list = ["growth_driver_l4", "growth_driver_l5"]
df = create_final_df("growth_driver_l4")

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

In [ ]:
# If utils exists with the needed encoded functions
from utils import *
# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))


In [ ]:
coord_to_columns = load.CoordToColumns(
    time='date',
    # geo='geo',
    # population='population',
    kpi='recrutements_hebdo', # choose the target variable you want
    revenu_per_kpi= 'revenu_per_kpi',
    controls= ['jour_ferie_wo_pentecote',
                'jour_ferie'
                'lundi_pentecote',
                'jour_ferie_dimanche',
                "start_of_month",
                "september",
                "may",
                'pont',
                'ferie_prolonge',
                'school_holiday',
                'end_of_year',
                'pre_end_of_year',
                'rentree_scolaire',
                'xmas_new_year',
                'back_to_school',
                'adjacent_back_to_school',
                'BOUYGUES_TEL_B_YOU',
                'BOUYGUES_TEL_BBOX',
                'BOUYGUES_TELECOM',
                'ORANGE',
                'ORANGE_LIVEBOX',
                'ORANGE_SOSH',
                'SFR_RED',
                'SFR',
                "sortie_ultra"
                ],
    non_media_treatments= [
                            ],
    media= [
            'nb_impressions_campaign_type_demand_gen',
            'nb_impressions_campaign_type_search',
            'nb_impressions_campaign_type_video',
            'nb_impressions_campaign_type_search',
            'nb_impressions_metadisplay',
            'nb_impressions_metavideo',
        ],
    media_spend=[
                'spend_campaign_type_demand_gen',
                'spend_campaign_type_search',
                'spend_campaign_type_video',
                'spend_campaign_type_pmax',
                'spend_metadisplay',
                'spend_metavideo',
                ],
    reach=['reach_tvfixe',
                'reach_tvmobile',
                'reach_ooh'],
    frequency=['frequency_tvfixe',
                    'frequency_tvmobile',
                    'frequency_ooh']
    rf_spend=['spend_tv_fixe',
                'spend_tv_mobile',
                'spend_ooh'],
)

In [ ]:
media_to_channel={
    'nb_impressions_metadisplay':'meta display',
    'nb_impressions_metavideo':'meta video',
    'nb_impressions_campaign_type_demand_gen' : 'google demande gen',
    'nb_impressions_campaign_type_display': 'google display',
    'nb_impressions_campaign_type_pmax':'google pmax',
    'nb_impressions_campaign_type_search':'google search', #on le met de cote pour le moment car va avoir tendance à cannibaliser les ventes
    'nb_impressions_campaign_type_video':'google video',
}
media_spend_to_channel={
    'spend_metadisplay':'meta display',
    'spend_metavideo':'meta video',
    'spend_metamixed':'meta mixed',
    'spend_campaign_type_demand_gen' : 'google demande gen',
    'spend_campaign_type_display': 'google display',
    'spend_campaign_type_pmax':'google pmax',
    'spend_campaign_type_search':'google search',
    'spend_campaign_type_shopping':'google shopping',
    'spend_campaign_type_video':'google video',
}
reach_to_channel={
            'reach_tv_fixe':'tv_fixe',
            'reach_tv_mobile':'tv_mobile',
            'reach_ooh':'ooh'
            }
frequency_to_channel={
            'frequency_tv_fixe':'tv_fixe',
            'frequency_tv_mobile':'tv_mobile',
            'frequency_ooh':'ooh'
    }
rf_spend_to_channel={
            'spend_tv_fixe':'tv_fixe',
    'spend_tv_mobile':'tv_mobile',
            'spend_ooh':'ooh'
    }

loader = load.DataFrameDataLoader(
    df=df, # Attention, le csv doit être au même endroit que le script python
    kpi_type='non_revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=media_to_channel,
    media_spend_to_channel=media_spend_to_channel,
    reach_to_channel=reach_to_channel,
    frequency_to_channel= frequency_to_channel,
    rf_spend_to_channel= rf_spend_to_channel,
)

In [ ]:
# Latence et saturation des canaux média  |  Meridian  |  Google for Developers
alpha_prior = tfp.distributions.Uniform(
low= [0.0001, 0.0001, 0.0001, 0.0001, 0.0001,0.0001],
high=[0.4, 0.3, 0.3, 0.6, 0.3, 0.4],
name=constants.ALPHA_M
)

# Elasticité cumulée, ie expected per euro spent during the period
ec_priors = tfp.distributions.TruncatedNormal(
    loc= [0.1, 0.2, 0.2,0.3, 0.4, 0.2],
    scale=[2.0, 2.0, 2.0, 2.0, 2.0, 2.0],
    low = 0.001,
    high= 10.0,
    name=constants.EC_M
)

# Intensity of media effect
# Google → strong direct effet, TV → harder to measure its effect through time
beta_prior = tfp.distributions.LogNormal(
    loc= [0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
scale=[0.17, 0.2, 0.2, 0.13, 0.17, 0.17]
, # pour jouer sur l'incertitude
    name=constants.BETA_M
)

alpha_prior_rf = tfp.distributions.Uniform(
low= [0.5, 0.5, 0.001],
high=[1.0, 1.0, 0.7],
name=constants.ALPHA_M
)

ec_priors_rf = tfp.distributions.TruncatedNormal(
    loc= [0.4, 0.4, 0.3],
    scale=[2.0, 2.0, 2.0],
    low = 0.001,
    high= 10.0,
    name=constants.EC_M
)

beta_prior_rf = tfp.distributions.LogNormal(
    loc= [0.0, 0.0, 0.0],
scale=[0.095, 0.09, 0.11]
, # pour jouer sur l'incertitude
    name=constants.BETA_M
)

knot_prior = tfp.distributions.Normal(
    loc = 0.0,
    scale = 0.2,
    name=constants.KNOT_VALUES)
# How much the baseline of each geo is allowed to changed compared to the one with the most population
baseline_prior = tfp.distributions.Normal(
		loc = 0.0,
		scale = 0.01,
		name = constants.BASELINE_GEO
)
prior = prior_distribution.PriorDistribution(
    knot_values=knot_prior,
    tau_g_excl_baseline=baseline_prior,
    alpha_m=alpha_prior,
    alpha_rf = alpha_prior_rf,
    beta_m=beta_prior,
    beta_rf = beta_prior_rf,
    ec_m=ec_priors,
    ec_rf = ec_priors_rf
)

In [ ]:
model_spec = spec.ModelSpec(knots = 20,
														prior=prior,
                            media_prior_type='coefficient',
                            rf_prior_type = 'coefficient',
                            baseline_geo = None,
                            max_lag= 18)

mmm =model.Meridian(
    input_data=df,
    model_spec=model_spec
)

mmm.sample_prior(500)
mmm.sample_posterior(n_chains=10, n_adapt=2000, n_burnin=500, n_keep=1000, seed=1)